The goal is to see if we can train a xgboost model to recognize what tweeet has a genuine location and what tweet doesn't. First step is to decide what set of tweets will be put into the model: the more we prune before we feed into the model the less we have to worry about the imbalance problem with training these models.



In [8]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import sys
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
import os
password = os.getenv('db_password')
engine = create_engine('postgres://%s:%s@104.154.139.71/%s'%('postgres',password,'tweets_db'))
connection = engine.connect()


None


In [36]:
df4 = pd.read_sql_query("""
select 
    tweets.tweet_id,
    tweet_date, 
    tweets.tweet_text, 
    entityname,
    tags
from tweets 
join osmtweetlocalization 
    on osmtweetlocalization.tweet_id = tweets.tweet_id 
join planet_osm_nodes
    on osmtweetlocalization.osm_id = planet_osm_nodes.id
join spacyentities01 
    on spacyentities01.tweet_id = tweets.tweet_id 
where 
    trim(entityname) not in ( 
        select trim(entityname) 
        from spacyentities01 
        group by trim(entityname) 
        order by count(id) desc
        limit 100
    )
    and tweets.tweet_id in (
        select tweet_id 
        from osmtweetlocalization  
        group by tweet_id 
        having count(tweet_id) <5 
    )
order by random();
""",engine)

In [37]:
print(len(df4))
for i in range(1,100):
    print(df4.tweet_text[i])

15096
It's official: Hurricane Warning. @CBS12 @ Embassy Suites by Hilton… https://t.co/zW4MTuueQ2
It's official: Hurricane Warning. @CBS12 @ Embassy Suites by Hilton… https://t.co/zW4MTuueQ2
Hurricane Warning ⚠️ from Jupiter Inlet south around the FL peninsula to Bonita Beach, as well as for the Keys, Lake Okeechobee &amp; FL Bay.
Hurricane Warning ⚠️ from Jupiter Inlet south around the FL peninsula to Bonita Beach, as well as for the Keys, Lake Okeechobee &amp; FL Bay.
Greenfield PD Helping in "Hurricane Harvey" Relief | Buckeye Country 105.5 https://t.co/E7lYEH0UJn
Greenfield PD Helping in "Hurricane Harvey" Relief | Buckeye Country 105.5 https://t.co/E7lYEH0UJn
Calm before the storm @ Indian Harbour Beach, Florida https://t.co/bk6iRGrPSJ
Sin Luz, Las Carmelitas, Calle San Jorge @AEEONLINE #Santurce #Irma #SinLuz #SanJuan
Sin Luz, Las Carmelitas, Calle San Jorge @AEEONLINE #Santurce #Irma #SinLuz #SanJuan
Sin Luz, Las Carmelitas, Calle San Jorge @AEEONLINE #Santurce #Irma #SinLuz #S

So this metric seems to give 7,000 tweets out of which a good fraction are actual location tweets. Lets try to train which of these tweets have actual location and which ones don't.

In [23]:
# create a table to store this training data.
connection.execute("""
create table tweetsTrainLocation01 (
id serial,
tweet_id bigint unique references tweets(tweet_id),
isALocationTweet boolean
);
""")
# table already created, running this will only give an error now.

In [28]:
connection.executeo("""
insert into tweetsTrainLocation01 (tweet_id)  (
    select 
        tweets.tweet_id
    from tweets 
    join osmtweetlocalization 
        on osmtweetlocalization.tweet_id = tweets.tweet_id 
    join planet_osm_nodes
        on osmtweetlocalization.osm_id = planet_osm_nodes.id
    join spacyentities01 
        on spacyentities01.tweet_id = tweets.tweet_id 
    where 
        trim(entityname) not in ( 
            select trim(entityname) 
            from spacyentities01 
            group by trim(entityname) 
            order by count(id) desc
            limit 100
        )
        and tweets.tweet_id in (
            select tweet_id 
            from osmtweetlocalization  
            group by tweet_id 
            having count(tweet_id) <5 
        )
) on conflict do nothing;
""")

The interactive part below Will let you keep Marking tweets as containing a location and that it pertains to Hurricane Irma, and you can keep training further whenever you have time!

In [88]:
from ipywidgets import widgets
from IPython.display import clear_output

button = widgets.Button(description="Yes location details.")
display(button)
button2 = widgets.Button(description="No location details.")
display(button2)
tweetid = 0
def on_yes_button(b):
    update_and_refresh("TRUE")
def on_no_button(b):
    update_and_refresh("FALSE")
def update_and_refresh(flag):
    global tweetid
    connection.execute("UPDATE tweetsTrainLocation01 SET isALocationTweet="+flag+" WHERE tweet_id = "+str(tweetid))
    getATweet()
def getATweet():
    tweetres = pd.read_sql_query("""
    select 
        tweets.tweet_text,tweets.tweet_id 
    from tweets 
    join tweetsTrainLocation01
    on 
        tweets.tweet_id = tweetsTrainLocation01.tweet_id
    where isALocationTweet is null order by random() limit 1;""",engine)
    global tweetid
    tweetid = tweetres.tweet_id[0]
    clear_output()
    print(tweetres.tweet_text[0])
getATweet()
button.on_click(on_yes_button)
button2.on_click(on_no_button)


Tropic Terrace Condo Hotel Unit #5 Treasure Island, Florida
(Photo Credits To The Owner) https://t.co/ukmM1MlC3k


In [100]:
pd.read_sql_query("select isALocationTweet,count(tweet_id) from tweetsTrainLocation01  group by isALocationTweet;",engine)


,isalocationtweet,count
0,None,6611
1,True,140
2,False,128
